In [152]:
#This is the functional Version. Use this for creating the report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [153]:
#disable warnings
import warnings; warnings.simplefilter('ignore')

The source files for this script are SalesForce reports Interactions.Contacts.csv and Interactions.Opportunity.Term.csv. The source files have to be stored in a NON-GIT folder so that they will never be available to git hub. Whenever the source files are updated into box the old source files should be placed in a depreciated state in a different folder. This script therefore will always pull the most recent Salesforce report results.


In [154]:
#INT is the contact information for INTERACTIONS
INT=pd.read_csv('C:/Users/mjc341/Desktop/UMAN 1507 Monthly INQ summary Report/Interactions.Contacts.csv',skipfooter=5,encoding='latin-1',engine ='python')
#INT=pd.read_csv('C:/Users/sxh706/Desktop/Interactions.by.Month/2018/June/Interactions.Contacts.csv',skipfooter=5,encoding='latin-1',engine ='python')

Please Use a new dataframe name if you are going to break any code



In [155]:
#CON is the CONTACT TERM information
CON=pd.read_csv('C:/Users/mjc341/Desktop/UMAN 1507 Monthly INQ summary Report/Interactions.Opportunity.Term.csv',skipfooter=5,encoding='latin-1',engine ='python')
#CON=pd.read_csv('C:/Users/sxh706/Desktop/Interactions.by.Month/2018/June/Interactions.Opportunity.Term.csv',skipfooter=5,encoding='latin-1',engine ='python')

In [156]:
len(INT)

1305259

In [157]:
len(CON)

818555

In [158]:
#INT.info()

In [159]:
INT.columns

Index(['Contact: Contact ID (18-digit)', 'Interaction: Interaction Name',
       'Admit Type', 'Contact: First Name', 'Contact: Middle Name',
       'Contact: Last Name', 'Contact: Ethnic Group', 'Contact: Ethnicity',
       'Contact: Gender', 'Term', 'Lead Source', 'Interaction: Created Date',
       'Contact: Mailing Street', 'Contact: Mailing City',
       'Contact: Mailing State/Province', 'Contact: Mailing Zip/Postal Code',
       'Contact: Mailing Country', 'Contact: Other City',
       'Contact: Other State/Province', 'Contact: Other Zip/Postal Code',
       'Contact: Other Country', 'Contact: Market Segment Code',
       'Contact: Market Segment'],
      dtype='object')

In [160]:
CON.columns

Index(['Contact ID (18-digit)', 'Full Name', 'Opportunity Name',
       'Term: Term Name', 'Opportunity Record Type', 'Inquiry', 'Inquiry Date',
       'Opportunity ID (18-digit)', 'Empl ID', 'Application Number'],
      dtype='object')

In [161]:
#CON.info()

In [162]:
#remove columns with identifing info
CON = CON.drop(columns =['Full Name'])

In [164]:
#remove columsn with identifing info
INT = INT.drop(columns= ['Contact: First Name', 'Contact: Middle Name',
       'Contact: Last Name'])

KeyError: "['Contact: First Name' 'Contact: Middle Name' 'Contact: Last Name'] not found in axis"

In [165]:
#changed the name of columns using a dictionary
INT = INT.rename(columns ={'Contact: Contact ID (18-digit)':'Contact_ID','Interaction: Interaction Name':'Int_Name',
       'Admit Type':'Admit_Type','Contact: Ethnic Group': 'Ethnic_Group', 'Contact: Ethnicity':'Ethnicity',
       'Contact: Gender':'Gender', 'Term':'Term', 'Lead Source':'Lead_Source', 'Interaction: Created Date':'Create_Date',
       'Contact: Mailing Street':'Mailing_Street', 'Contact: Mailing City':'Mailing_City',
       'Contact: Mailing State/Province':'Mailing_State', 'Contact: Mailing Zip/Postal Code':'Mailing_Postal',
       'Contact: Mailing Country':'Mailing_Country', 'Contact: Other City':'Other_City',
       'Contact: Other State/Province':'Other_State', 'Contact: Other Zip/Postal Code':'Other_Postal',
       'Contact: Other Country':'Mailing_Country', 'Contact: Market Segment Code':'Market_Segment_Code',
       'Contact: Market Segment':'Market_Segment'},inplace =False)

In [166]:
#changed the name of columns
CON = CON.rename(columns={  'Contact ID (18-digit)':'Contact_ID', 'Opportunity Name':'OppName', 
       'Term: Term Name':'CONTerm', 'Opportunity Record Type':'Record_Type', 'Inquiry':'Inquiry', 'Inquiry Date':'Inquiry_Date',
       'Opportunity ID (18-digit)':'Opp_ID', 'Empl ID':'EMPL', 'Application Number':'App_Number'})

In [167]:
#Sort INT by Contact_ID and Interaction Name
INT = INT.sort_values(by = ['Contact_ID','Int_Name'])

In [168]:
#Sort CON by Contact_ID and Opportunity Name
CON = CON.sort_values(by = ['Contact_ID','OppName'])

In [169]:
#Dropping the duplicates to then merge later
CON=CON.drop_duplicates(['Contact_ID'], keep='first')

In [170]:
#Inner Join
INNER_INT = pd.merge(left = INT, right = CON, left_on= 'Contact_ID', right_on = 'Contact_ID')

In [171]:
len(INNER_INT)

1084776

INT has been MERGED with CON on Contact_ID this will allow us to find one TERM for each contact ID

In [172]:
INNER_INT.head(1)

,Contact_ID,Int_Name,Admit_Type,Ethnic_Group,Ethnicity,Gender,Term,Lead_Source,Create_Date,Mailing_Street,...,Market_Segment_Code,Market_Segment,OppName,CONTerm,Record_Type,Inquiry,Inquiry_Date,Opp_ID,EMPL,App_Number
0,0031500001d4Si8AAE,20160320-0100599,New Student,NaN,NaN,Female,Fall 2015,Purchased List,1/4/2013,12831 sw 43 dr,...,NaN,NaN,fernandez - Fall 2015,Fall 2015,UGrad Inquiry,1.0,3/20/2016,0061500000Sr5YQAAZ,NaN,NaN


In [173]:
# Create a datetime variable from Create_Date this operation takes up to 3 min
#INT['Create_Date']= pd.to_datetime(INT.Create_Date)
INNER_INT['Create_Date']= pd.to_datetime(INNER_INT.Create_Date)

In [20]:
#INT.info()

In [174]:
#find earliest create date
#INT['Create_Date'].min()
INNER_INT['Create_Date'].min()

Timestamp('2001-08-06 00:00:00')

Here we will insert the lambda function to extract the YEAR and MONTH from the new datetime type so we can filter on YEAR and groupby MONTH also add M col that can be dictionaried to Names of MONTHS instead of numbers. Question can we format the month names at the conversion?

In [175]:
#INT.loc[:,'YEAR'] =INT['Create_Date'].apply(lambda x: "%d" % (x.year))
INNER_INT.loc[:,'YEAR'] =INNER_INT['Create_Date'].apply(lambda x: "%d" % (x.year))

In [176]:
#INT.loc[:,'MONTH'] = INT['Create_Date'].apply(lambda x: "%d" % (x.month))
INNER_INT.loc[:,'MONTH'] = INNER_INT['Create_Date'].apply(lambda x: "%d" % (x.month))

In [177]:
INNER_INT.head(1)

,Contact_ID,Int_Name,Admit_Type,Ethnic_Group,Ethnicity,Gender,Term,Lead_Source,Create_Date,Mailing_Street,...,OppName,CONTerm,Record_Type,Inquiry,Inquiry_Date,Opp_ID,EMPL,App_Number,YEAR,MONTH
0,0031500001d4Si8AAE,20160320-0100599,New Student,NaN,NaN,Female,Fall 2015,Purchased List,2013-01-04,12831 sw 43 dr,...,fernandez - Fall 2015,Fall 2015,UGrad Inquiry,1.0,3/20/2016,0061500000Sr5YQAAZ,NaN,NaN,2013,1


In [178]:
# filter on term for 2017,2018,2019,2020
#TERM2017 = INT.loc[(INT.Term == 'Fall 2017')]
#TERM2018 = INT.loc[(INT.Term == 'Fall 2018')]
#TERM2019 = INT.loc[(INT.Term == 'Fall 2019')]
#TERM2020 = INT.loc[(INT.Term == 'Fall 2020')]
TERM2017 = INNER_INT.loc[(INNER_INT.Term == 'Fall 2017')]
TERM2018 = INNER_INT.loc[(INNER_INT.Term == 'Fall 2018')]
TERM2019 = INNER_INT.loc[(INNER_INT.Term == 'Fall 2019')]
TERM2020 = INNER_INT.loc[(INNER_INT.Term == 'Fall 2020')]

In [179]:
len(TERM2017)

182427

In [180]:
len(TERM2018)

143300

In [181]:
len(TERM2019)

80331

In [182]:
len(TERM2020)

24267

In [183]:
#filter on YEAR for  2016,2017,2018,2019
#INQ2016 df is inquiries made in 2016 for 2017 TERM
INQ2016 = TERM2017[TERM2017['YEAR'] == '2016']
INQ2017 = TERM2018[TERM2018['YEAR'] == '2017']
INQ2018 = TERM2019[TERM2019['YEAR'] == '2018']
INQ2019 = TERM2020[TERM2020['YEAR'] == '2019']

In [184]:
len(INQ2016)

100684

In [185]:
len(INQ2017)

86787

In [186]:
len(INQ2018)

52911

In [187]:
len(INQ2019)

0

In [188]:
#show head of INQ2016 same for 2017,2018,2019
INQ2016.head(1)

,Contact_ID,Int_Name,Admit_Type,Ethnic_Group,Ethnicity,Gender,Term,Lead_Source,Create_Date,Mailing_Street,...,OppName,CONTerm,Record_Type,Inquiry,Inquiry_Date,Opp_ID,EMPL,App_Number,YEAR,MONTH
8,0031500001d4SiCAAU,20160320-0910984,New Student,NaN,Black/African American,Female,Fall 2017,Web Form,2016-01-12,3 Horsley Road,...,Tetteh - Fall 2014,Fall 2014,UGrad Applicant,0.0,NaN,0061500000T7M50AAF,50442768.0,98893.0,2016,1


In [189]:
INQ2016.columns

Index(['Contact_ID', 'Int_Name', 'Admit_Type', 'Ethnic_Group', 'Ethnicity',
       'Gender', 'Term', 'Lead_Source', 'Create_Date', 'Mailing_Street',
       'Mailing_City', 'Mailing_State', 'Mailing_Postal', 'Mailing_Country',
       'Other_City', 'Other_State', 'Other_Postal', 'Mailing_Country',
       'Market_Segment_Code', 'Market_Segment', 'OppName', 'CONTerm',
       'Record_Type', 'Inquiry', 'Inquiry_Date', 'Opp_ID', 'EMPL',
       'App_Number', 'YEAR', 'MONTH'],
      dtype='object')

In [190]:
#confirm that filtered data is a datafram
type(INQ2016)

pandas.core.frame.DataFrame

In [191]:
#pd.show_versions()

In [192]:
#get count by month of each term
INQ2016.Create_Date.dt.month.value_counts().sort_index()

1      3501
2      4667
3      1686
4      1412
5      2547
6     12482
7     12931
8      2435
9      7711
10    18401
11    17755
12    15156
Name: Create_Date, dtype: int64

In [193]:
INQ2017.Create_Date.dt.month.value_counts().sort_index()

1      1882
2      3794
3      3612
4      3557
5     10571
6      6944
7      3798
8      4991
9      5989
10    20878
11    15739
12     5032
Name: Create_Date, dtype: int64

In [194]:
INQ2018.Create_Date.dt.month.value_counts().sort_index()

1     2418
2     2754
3     9018
4    11967
5     7052
6    11067
7     6753
8     1882
Name: Create_Date, dtype: int64

In [195]:
#this creates an empty list because no INQ can be found for 2019
#INQ2019.Create_Date.dt.month.value_counts().sort_index()

In [196]:
#create a timestamp to get values from the FAll 2018 term that are prior to Jan 1 2017
tsprior = pd.Timestamp('1/1/2016')
tsprior2 = pd.Timestamp ('1/1/2017')
tsprior3 = pd.Timestamp('1/1/2018')
tsprior4 = pd.Timestamp('1/1/2019')

In [197]:
tsprior2

Timestamp('2017-01-01 00:00:00')

In [198]:
B4JAN16 = TERM2017.loc[TERM2017.Create_Date <= tsprior,:]
B4JAN17 = TERM2018.loc[TERM2018.Create_Date <= tsprior2,:]
B4JAN18 = TERM2019.loc[TERM2019.Create_Date <= tsprior3,:]
#B4JAN19 = TERM2020.loc[TERM2019.Create_Date <= tsprior4,:]

In [199]:
len(B4JAN16)

58709

In [200]:
len(B4JAN17)

35755

In [201]:
len(B4JAN18)

27462

In [202]:
len(B4JAN16)

58709

In [203]:
#test the counts
B4JAN16['Contact_ID'].value_counts().sum()
#B4JAN17['Contact_ID'].value_counts().sum()
#B4JAN18['Contact_ID'].value_counts().sum()

58709

In [204]:
#B4JAN19['Contact_ID'].value_counts().sum()

In [205]:
R16 = INQ2016.Create_Date.dt.month.value_counts().sort_index()
R17 = INQ2017.Create_Date.dt.month.value_counts().sort_index()
R18 = INQ2018.Create_Date.dt.month.value_counts().sort_index()
R19 = INQ2019.Create_Date.dt.month.value_counts().sort_index()

In [206]:
R16df = pd.DataFrame([R16]).T
R17df = pd.DataFrame([R17]).T
R18df = pd.DataFrame([R18]).T
R19df = pd.DataFrame([R19]).T

In [207]:
#R16df.head()

In [208]:
a =R16df.rename({'Create_Date': 'Count-2016'}, axis = 'columns')
#R16df.rename({'Create_Date': 'Count-2016'}, axis = 'columns')

In [209]:
b = R17df.rename({'Create_Date': 'Count-2017'}, axis = 'columns')
#R17df.rename({'Create_Date': 'Count-2017'}, axis = 'columns')

In [210]:
#R18df.rename({'Create_Date': 'Count-2018'}, axis = 'columns')
c = R18df.rename({'Create_Date': 'Count-2018'}, axis = 'columns')

In [211]:
#d = R19df.rename({'Create_Date': 'Count-2019'}, axis = 'columns')

In [212]:
 j = pd.concat([a, b], axis=1, join_axes=[a.index])
    

In [213]:
#j.head(1)

In [214]:
k = pd.concat([j, c], axis=1, join_axes=[a.index])

In [215]:
#k2 = pd.concat([k, d], axis=1, join_axes=[a.index])

In [216]:
k.head()

,Count-2016,Count-2017,Count-2018
1,3501,1882,2418.0
2,4667,3794,2754.0
3,1686,3612,9018.0
4,1412,3557,11967.0
5,2547,10571,7052.0


In [217]:
#2019 has no results so do not include d
#k2.head()

In [218]:
type(k)

pandas.core.frame.DataFrame

In [219]:
k.dtypes

Count-2016      int64
Count-2017      int64
Count-2018    float64
dtype: object

In [220]:
k =  k.replace(np.nan,0)

In [221]:
k['Count-2018'] = k['Count-2018'].astype(int)
k

,Count-2016,Count-2017,Count-2018
1,3501,1882,2418
2,4667,3794,2754
3,1686,3612,9018
4,1412,3557,11967
5,2547,10571,7052
6,12482,6944,11067
7,12931,3798,6753
8,2435,4991,1882
9,7711,5989,0
10,18401,20878,0


In [222]:
#k.dtypes

In [223]:
k.index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64')

In [224]:
# create  a dictionary to map the month values
dict = {1:'JAN',2:'FEB',3:'MAR',4:'APR',5:'MAY',6:'JUN',7:'JUL',8:'AUG',9:'SEP',10:'OCT',11:'NOV',12:'DEC'}

In [225]:
#map the dictionary to the new col on index
k ['MONTH'] = k.index.map(dict)

In [226]:
k

,Count-2016,Count-2017,Count-2018,MONTH
1,3501,1882,2418,JAN
2,4667,3794,2754,FEB
3,1686,3612,9018,MAR
4,1412,3557,11967,APR
5,2547,10571,7052,MAY
6,12482,6944,11067,JUN
7,12931,3798,6753,JUL
8,2435,4991,1882,AUG
9,7711,5989,0,SEP
10,18401,20878,0,OCT


In [227]:
#change the name of the df and rearragne the cols
INQbyMONTH = k[['MONTH','Count-2016','Count-2017','Count-2018']]

In [228]:
#this is the result
INQbyMONTH

,MONTH,Count-2016,Count-2017,Count-2018
1,JAN,3501,1882,2418
2,FEB,4667,3794,2754
3,MAR,1686,3612,9018
4,APR,1412,3557,11967
5,MAY,2547,10571,7052
6,JUN,12482,6944,11067
7,JUL,12931,3798,6753
8,AUG,2435,4991,1882
9,SEP,7711,5989,0
10,OCT,18401,20878,0


In [229]:
#it is a df
type(INQbyMONTH)

pandas.core.frame.DataFrame

In [230]:
x = B4JAN16['Contact_ID'].value_counts().sum()
y = B4JAN17['Contact_ID'].value_counts().sum()
z = B4JAN18['Contact_ID'].value_counts().sum()
#xx = B4JAN19['Contact_ID'].value_counts().sum()

In [231]:
PRIOR = pd.DataFrame({'Prior to Jan 2016': x , 'Prior to Jan 2017': y , 'Prior to Jan 2018': z},index = [0])

In [232]:
PRIOR

,Prior to Jan 2016,Prior to Jan 2017,Prior to Jan 2018
0,58709,35755,27462


In [233]:
#create a timestamp to get prior values for FEB and AUG, JUN
tspriorFEB17 = pd.Timestamp('2/1/2017')
tspriorFEB18 = pd.Timestamp ('2/1/2018')
tspriorFEB19 = pd.Timestamp('2/1/2019')
tspriorAUG17 = pd.Timestamp('8/1/2017')
tspriorAUG18 = pd.Timestamp ('8/1/2018')
tspriorAUG19 = pd.Timestamp('8/1/2019')
tsJUN312017 = pd.Timestamp('6/30/2017')
tsJUN312018 = pd.Timestamp('6/30/2018')
tsJUN312019 = pd.Timestamp('6/30/2019')

In [234]:
#LeadSource Counts for AdmitTerm 2017 that occured in 2016
LEAD17 = INQ2016.pivot_table(index=['Lead_Source'],values =['Contact_ID'],aggfunc =len)
LEAD17.rename(index= str, columns={'Contact_ID':'2017'},inplace =True)
LEAD17

,2017
Lead_Source,
College Fair,7782
College Visit,1
Education Fair,3
Email,116
Event Registration,8072
Group Tour,252
High School Visit,5427
Initiated Application,9010
International Travel,1


In [235]:
#LeadSource Counts for AdmitTerm 2018 that occured in 2017
LEAD18 = INQ2017.pivot_table(index=['Lead_Source'],values =['Contact_ID'],aggfunc =len)
LEAD18.rename(index= str, columns={'Contact_ID':'2018'},inplace =True)
LEAD18

,2018
Lead_Source,
College Fair,5205
College Visit,12
Education Fair,2
Email,24
Event Registration,9130
Group Tour,278
High School Visit,5410
Initiated Application,8
Lead Card,175


In [236]:
#LeadSource Counts for AdmitTerm 2019 that occured in 2018
LEAD19 = INQ2018.pivot_table(index=['Lead_Source'],values =['Contact_ID'],aggfunc =len)
LEAD19.rename(index= str, columns={'Contact_ID':'2019'},inplace =True)
LEAD19

,2019
Lead_Source,
College Fair,5604
Education Fair,10
Email,8
Event Registration,8606
Group Tour,206
High School Visit,42
Lead Card,145
Office Visit,236
Phone,1


In [237]:
Lead1 = pd.concat([LEAD17, LEAD18], axis=1, join_axes=[LEAD17.index])
Lead1

,2017,2018
Lead_Source,,
College Fair,7782,5205.0
College Visit,1,12.0
Education Fair,3,2.0
Email,116,24.0
Event Registration,8072,9130.0
Group Tour,252,278.0
High School Visit,5427,5410.0
Initiated Application,9010,8.0
International Travel,1,NaN


In [238]:
Lead2 = pd.concat([Lead1, LEAD19], axis=1, join_axes=[Lead1.index])
Lead2

,2017,2018,2019
Lead_Source,,,
College Fair,7782,5205.0,5604.0
College Visit,1,12.0,NaN
Education Fair,3,2.0,10.0
Email,116,24.0,8.0
Event Registration,8072,9130.0,8606.0
Group Tour,252,278.0,206.0
High School Visit,5427,5410.0,42.0
Initiated Application,9010,8.0,NaN
International Travel,1,NaN,NaN


In [239]:
LEAD = Lead2.fillna(0)
LEAD['2018'] = LEAD['2018'].astype(int)
LEAD['2019'] = LEAD['2019'].astype(int)
LEAD

,2017,2018,2019
Lead_Source,,,
College Fair,7782,5205,5604
College Visit,1,12,0
Education Fair,3,2,10
Email,116,24,8
Event Registration,8072,9130,8606
Group Tour,252,278,206
High School Visit,5427,5410,42
Initiated Application,9010,8,0
International Travel,1,0,0


In [240]:
#Count all Inq for the 2017 Admit term that occured before August of 2017
B4AUG17 = TERM2017.loc[TERM2017.Create_Date <= tspriorAUG17,:]
len(B4AUG17)

181169

In [241]:
#Count all Inq for the 2018 Admit term that occured before August of 2018
B4AUG18 = TERM2018.loc[TERM2018.Create_Date <= tspriorAUG18,:]
len(B4AUG18)

143238

In [242]:
# create the path where the results willbe sent
path_final = r'C:/Users/mjc341/Desktop/EMSA-GIT/COLAB'

In [243]:
path_final

'C:/Users/mjc341/Desktop/EMSA-GIT/COLAB'

In [244]:
writer = pd.ExcelWriter(path_final + '/' + 'SalesForceByMonth1' + '.xlsx')
INQbyMONTH.to_excel(writer,'Results')
PRIOR.to_excel(writer, 'PRIOR')
LEAD.to_excel(writer,'LEAD')
workbook = writer.book
worksheet = writer.sheets['Results'] 
worksheet2 = writer.sheets ['PRIOR']
worksheet3 = writer.sheets ['LEAD']

format1 = workbook.add_format({'num_format': '#,##', 'bold': True, 'border': '1'})
format2 = workbook.add_format({'num_format': '##,###', 'border': '1','bg_color':'#FFC7CE'})
format3 = workbook.add_format({'border': '1','bg_color':'#DAF7A6'})
format4 = workbook.add_format({'border': '1','bg_color':'#DAF7A6'})
#added bkground color so we can show different years

# Note: It isn't possible to format any cells that already have a format such
# as the index or headers or any cells that contain dates or datetimes.

worksheet.set_column('A:A', 10)
worksheet.set_column('B:E', 12)
worksheet.set_row(1, None, format2)
worksheet.set_row(2, None, format2)
worksheet.set_row(3, None, format2)
worksheet.set_row(4, None, format2)
worksheet.set_row(5, None, format2)
worksheet.set_row(6, None, format2)
worksheet.set_row(7, None, format2)
worksheet.set_row(8, None, format2)
worksheet.set_row(9, None, format2)
worksheet.set_row(10, None, format2)
worksheet.set_row(11, None, format2)
worksheet.set_row(12, None, format2)

worksheet2.set_column('A:A', 10)
worksheet2.set_column('B:E', 18)
worksheet2.set_row(1, None, format3)
worksheet2.set_row(2, None, format4)

worksheet3.set_column('A:A', 10)
worksheet3.set_column('B:E', 18)
worksheet3.set_row(1, None, format3)
worksheet3.set_row(2, None, format4)
worksheet3.set_row(3, None, format4)
worksheet3.set_row(4, None, format4)
worksheet3.set_row(5, None, format4)
worksheet3.set_row(6, None, format4)
worksheet3.set_row(7, None, format4)
worksheet3.set_row(8, None, format4)
worksheet3.set_row(9, None, format4)
worksheet3.set_row(10, None, format4)
worksheet3.set_row(11, None, format4)
worksheet3.set_row(12, None, format4)
worksheet3.set_row(13, None, format4)
worksheet3.set_row(14, None, format4)
worksheet3.set_row(15, None, format4)
worksheet3.set_row(16, None, format4)
worksheet3.set_row(17, None, format4)
worksheet3.set_row(18, None, format4)
worksheet3.set_row(19, None, format4)
worksheet3.set_row(20, None, format4)


#dont write until you check the code then remove the # below
writer.save()